In [1]:
import os
import json
import re
from groq import Groq

In [2]:
output_dir = "output"
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [3]:
def call_groq(prompt):
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": [{"type": "text", "text": prompt}]}],
        model="llama3-8b-8192",
    )
    return chat_completion.choices[0].message.content

In [4]:
with open(os.path.join(output_dir, "microservice_splits.json"), "r") as f:
    microservices = json.load(f)

with open(os.path.join(output_dir, "file_descriptions.json"), "r") as f:
    file_descriptions = json.load(f)
    described_files = list(file_descriptions.keys())

In [5]:
prompt = f"""
You are an expert software architect.

Based on the following information:
1. Microservice splits for each original file:
{json.dumps(microservices, indent=2)}

2. List of all newly generated files after splitting:
{json.dumps(described_files, indent=2)}

Your task is to generate a **clean, organized folder and file structure** for a microservices-based architecture.

### Rules:
- Use **only the filenames** from the list of newly generated files.
- Use folders **only if the file paths suggest them clearly**, or if multiple files belong to a common module (e.g., `auth`, `db`, `utils`, etc.).
- If a file does not belong to a group, place it at the root level.
- Do **not** create new folders unless they are already implied in the file path or there's a clear grouping need.
- Output must be a tree structure using the following format.
- No additional text, no comments, no blank lines.

### Output Format (Markdown):
```
folder_name
  └── file_name.py
standalone_file.py
```

**Strictly follow the output format. Use exact file names and folder hints from the paths above.**
"""

In [6]:
def clean_code(response_text: str):
    response_text = response_text.strip()
    
    parts = response_text.split("```")
    if len(parts) > 1:
        response_text = parts[1].strip()
    
    if response_text.lower().startswith("python"):
        response_text = response_text.split("\n", 1)[-1].strip()
    
    if "Note that" in response_text:
        response_text = response_text.split("Note that", 1)[0].strip()
    
    response_text = response_text.replace("```", "").strip()
    
    return response_text

In [7]:
new_structure = call_groq(prompt)

output_path = os.path.join(output_dir, "new_microservice_structure.txt")
new_structure = clean_code(new_structure)
with open(output_path, "w", encoding="utf-8") as f:
    f.write(new_structure)

print(f"[INFO] New microservices structure saved to: {output_path}")

[INFO] New microservices structure saved to: output\new_microservice_structure.txt
